# Обработка сущностей из Викиданных и создание словарей

In [65]:
import re
from pymystem3 import Mystem
from string import punctuation

punct = punctuation+'«»—…“”*–'
morph = Mystem()

In [2]:
def read_tsv(file):
    with open(file, "r") as f:
        f = f.readlines()
    data = []
    for line in f:
        line = line.strip('\n')
        q, entity, aliases = line.split('\t')
        data.append([q, entity, aliases])
    return data

Читаем tsv-файл, в котором собраны сущности из русскоязычной версии Викиданных, их уникальные идентификаторы Q и другие возможные наименования. 

In [3]:
%%time
data = read_tsv("/Users/anyway/Desktop/wikidata/all-entities-bigfile.tsv")

CPU times: user 18.1 s, sys: 11.9 s, total: 30 s
Wall time: 35.7 s


In [4]:
len(data)

5837666

In [5]:
data[1226]

['Q11349', 'туманность Орёл', 'M 16|NGC 6611']

In [6]:
%%time
#чистим Викиданные
def clean_wikidata(data):
    russian = "[А-Яа-я]+"
    prefixes = ("Категория:", "Шаблон:", "Приложение:", "ВЭ /", "БЭАН /", 
                "ЭСБЕ /", "РБС /", "БЭЮ /", "ЭЛ /", "МЭСБЕ /", "ЕЭБЕ /",
                "ТСД /", "ТСД2 /", "ПБЭ /", "БСЭ1 /", "НЭС /", "ББСРП /", 
                "РСКД /", "ТСД3 /") 
    data = [i for i in data if len(i[1]) > 2 
            and re.search(russian, i[1]) 
            and not i[1].startswith(prefixes)]
    for indx, line in enumerate(data):
        aliases = line[2].split('|')
        data[indx][2] = " | ".join([i for i in aliases if re.search(russian, i)])
    with open('/Users/anyway/Desktop/wikidata/clean-russian-wikidata.tsv', 'w') as f:
        for i in data:
            f.write("\t".join(i) + '\n')
    return data

clean_data = clean_wikidata(data)    
len(clean_data)

CPU times: user 19.1 s, sys: 4.12 s, total: 23.2 s
Wall time: 24.5 s


In [7]:
print(len(clean_data))
clean_data[:100]

2234352


[['Q31', 'Бельгия', 'Королевство Бельгия'],
 ['Q8', 'счастье', ''],
 ['Q23', 'Джордж Вашингтон', 'Вашингтон, Джордж'],
 ['Q24', 'Джек Бауэр', ''],
 ['Q42', 'Дуглас Адамс', 'Адамс, Дуглас'],
 ['Q1868', 'Поль Отле', 'Отле, Поль'],
 ['Q2013', 'Викиданные', 'Викидата'],
 ['Q45', 'Португалия', 'Португальская Республика'],
 ['Q51', 'Антарктида', ''],
 ['Q58', 'пенис', 'детородный орган | половой член | член'],
 ['Q68', 'компьютер', ''],
 ['Q75', 'Интернет', ''],
 ['Q125', 'ноябрь', 'ноября'],
 ['Q140', 'лев', ''],
 ['Q144', 'собака', 'собака обыкновенная'],
 ['Q147', 'котёнок', 'Котёнок | Котенок'],
 ['Q148', 'Китайская Народная Республика', 'КНР | Китай | Поднебесная'],
 ['Q155', 'Бразилия', 'Федеративная Республика Бразилия'],
 ['Q163', 'Йоркшир', ''],
 ['Q177', 'пицца', ''],
 ['Q178', 'макаронные изделия', 'паста | макароны'],
 ['Q183', 'Германия', 'Федеративная Республика Германия | ФРГ | Дойчланд'],
 ['Q207', 'Джордж Уокер Буш', 'Джордж Буш - младший | Джордж Буш-младший'],
 ['Q210', 'п

In [53]:
def delete_stress(data):
    for i, line in enumerate(data):
        data[i][1] = line[1].replace(chr(769), '')
        data[i][2] = line[2].replace(chr(769), '')
    return data

clean_data = delete_stress(clean_data)

In [66]:
%%time

#лемматизируем Викиданные и почистим от лишней пунктуации
def normalize(data):
    norm_data = []
    w = open("/Users/anyway/Desktop/wikidata/norm-wikidata.tsv", "w")
    for line in data:
        entity = ''.join(morph.lemmatize(line[1])).strip()
        entity = ' '.join([word.strip(punct) for word in entity.split() if word])
        aliases = line[2].lower()
        if aliases != '':
            if ' | ' in aliases:
                aliases = [''.join(morph.lemmatize(alias)).strip() for alias in aliases.split(" | ")]
                aliases = [' '.join([word.strip(punct) for word in alias.split() if word]) 
                           for alias in aliases]
                norm_data.append([line[0], [entity] + aliases])
                w.write(line[0] + ' | ' + ' | '.join([words for words in [entity] + aliases]) + '\n')
            else:
                aliases = ''.join(morph.lemmatize(aliases)).strip()
                aliases = ' '.join([word.strip(punct) for word in aliases.split() if word])
                norm_data.append([line[0], [entity] + [aliases]])
                w.write(line[0] + ' | ' + ' | '.join(words for words in [entity] + [aliases]) + '\n')
        else:
            norm_data.append([line[0], entity])
            w.write(line[0] + ' | ' + entity + '\n')
    w.close()
    return norm_data

norm_data = normalize(clean_data)

CPU times: user 5min 43s, sys: 1min 53s, total: 7min 37s
Wall time: 23min 8s


Данная структура списка списков удобна для создания словарей. 

In [67]:
norm_data[500:550]

[['Q5257', 'сатавахан'],
 ['Q5259', '1926 год'],
 ['Q5275', 'астрономический часы'],
 ['Q5279', 'нахродт-виблингверде'],
 ['Q5285', 'тарлак'],
 ['Q5300',
  ['центральный процессор',
   'цп',
   'центральный процессорный устройство',
   'цпу']],
 ['Q5317', ['спейс нидло', 'космический игла']],
 ['Q5321', ['резистор', 'сопротивление']],
 ['Q5322', 'электрический конденсатор'],
 ['Q5329', ['децибел', 'дб']],
 ['Q5335', 'харм вирсма'],
 ['Q5351', 'гильом IX трубадур'],
 ['Q5363', 'мария вентадорнский'],
 ['Q5380', 'робер де борона'],
 ['Q5381', '1218 год'],
 ['Q5383', ['дэвид боуи', 'дэвид роберт джонс']],
 ['Q5386',
  ['автоспорт',
   'автомобильный спорт',
   'автогонки',
   'автомобильный гонка',
   'гонки']],
 ['Q5389', 'олимпийский игра'],
 ['Q5414', 'немецкий дог'],
 ['Q5417', '1231 год'],
 ['Q5449', ['йошкар-ола', 'царевококшайск', 'краснококшайск']],
 ['Q5453', 'искусство волшебство'],
 ['Q5454', 'изабелла теоточить альбрицци'],
 ['Q5482', '1247 год'],
 ['Q5500', '1254 год'],
 ['Q5